In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")
from importlib import reload

import datajoint as dj
from pathlib import Path

import datajoint_utils as du
du = reload(du)
from importlib import reload

import time

WARNING - 2020-11-16 06:46:23,090 - trimesh_repair - Need to pip install annotationframeworkclient to repair mesh with pychunkedgraph
WARNING - 2020-11-16 06:46:23,092 - trimesh_io - Need to pip install annotationframeworkclient to use dataset_name parameters
INFO - 2020-11-16 06:46:23,417 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-16 06:46:23,418 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-16 06:46:23,443 - connection - C

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-16 06:46:23,625 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-16 06:46:23,625 - settings - Setting database.user to celiib
INFO - 2020-11-16 06:46:23,626 - settings - Setting database.password to newceliipass
INFO - 2020-11-16 06:46:23,627 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-16 06:46:23,794 - settings - Setting enable_python_native_blobs to True


# configuring the virtual module

In [2]:
import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-16 06:46:23,810 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-16 06:46:23,811 - settings - Setting database.user to celiib
INFO - 2020-11-16 06:46:23,812 - settings - Setting database.password to newceliipass
INFO - 2020-11-16 06:46:23,814 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-16 06:46:23,994 - settings - Setting enable_python_native_blobs to True


Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron

In [4]:
#so that it will have the adapter defined
from datajoint_utils import *

In [5]:
@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation
    ---
    decomposition: <decomposition>
    n_error_limbs: int #the number of branches that are touching multiple somas or 1 soma in multiple places
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    n_spines: bigint
    skeletal_length: double
    spine_density=null:double
    max_limb_skeletal_length:double
    median_branch_length:double #gives information on average skeletal length to next branch point
    max_limb_n_branches:int
    """
    
    """
    to add:
    median_branch_length
    **branch_width data (both types)**
    - max
    - 90 percentile
    
    spines volumes/info
    spines per branch
    
    
    """

    key_source = minnie.Decimation & (minnie.BaylorSegmentCentroid() & "multiplicity>=2").proj()

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = key['version']
        print(f"\n\n----- Working on {segment_id}-------")

        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        #somas = du.get_soma_mesh_list(segment_id) 
        somas = None

        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Calculate all starter stats
        # ----------- calculating the attributes --------- #
        n_error_limbs = nru.error_limb_indexes(recovered_neuron)

        n_somas = recovered_neuron.get_soma_node_names()

        n_limbs = recovered_neuron.get_limb_node_names()

        n_branches_per_limb = [len(ex_limb.get_branch_names()) for ex_limb in recovered_neuron]
        n_branches = np.sum(n_branches_per_limb)

        n_spines = len(recovered_neuron.spines)

        #for total skeletal length
        sk_len_per_branch = [sk.calculate_skeleton_distance(limb.skeleton) for limb in recovered_neuron]
        skeletal_length = np.sum(sk_len_per_branch)

        if skeletal_length > 0:
            spine_density = n_spines/skeletal_length
        else:
            spine_density = 0

        max_limb_skeletal_length = np.max(sk_len_per_branch)

        max_limb_n_branches = np.max(n_branches_per_limb)


        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                        n_error_limbs=n_error_limbs,
                        n_somas=n_somas,
                        n_limbs=n_limbs,
                        n_branches=n_branches,
                        n_spines=n_spines,
                        skeletal_length=skeletal_length,
                        spine_density=spine_density,
                        max_limb_skeletal_length=max_limb_skeletal_length,
                        max_limb_n_branches=max_limb_n_branches
                      )
#         else:
#             ret_file_path_str = "/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135574982852_0.pbz2"
#             new_key = dict(key,
#                            decomposition=ret_file_path_str,
#                             n_error_limbs=0,
#                             n_somas=0,
#                             n_limbs=0,
#                             n_branches=0,
#                             n_spines=0,
#                             skeletal_length=0,
#                             spine_density=0,
#                             max_limb_skeletal_length=0,
#                             max_limb_n_branches=0
#                           )
        
        
        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

    

In [ ]:
minnie.Decimation & (minnie.BaylorSegmentCentroid() & "multiplicity>=2").proj()

In [ ]:
((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()


In [ ]:
# errored_keys = (schema.jobs & "table_name = '__decomposition'").fetch("key")#.delete()
# errored_keys

In [6]:
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import trimesh as tu
tu = reload(tu)

Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')

INFO - 2020-11-16 06:46:30,522 - autopopulate - Found 663 keys to populate
INFO - 2020-11-16 06:46:30,715 - connection - Transaction started
INFO - 2020-11-16 06:46:30,719 - autopopulate - Populating: {'segment_id': 864691135608655300, 'version': 0, 'decimation_ratio': Decimal('0.25')}




----- Working on 864691135608655300-------

--- Beginning preprocessing of 864691135608655300---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 3668 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/864691135608655300/neuron_864691135608655300.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/864691135608655300/neuron_864691135608655300_decimated.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/864691135608655300/decimation_meshlab_25475853.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(188621, 3), faces.shape=(378345, 3))>]
--


meshes_split = [<trimesh.Trimesh(vertices.shape=(3941, 3), faces.shape=(7688, 3))>, <trimesh.Trimesh(vertices.shape=(55, 3), faces.shape=(104, 3))>, <trimesh.Trimesh(vertices.shape=(6, 3), faces.shape=(4, 3))>]
meshes_split_sdf = [0.8238685  0.00295772 0.2822315 ]
removing mesh interior before segmentation
xvfb-run -n 7011 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_27322.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_27322_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/temp/fill_holes_892517.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files
Error While parsing the XML filter plugin descriptors:


meshes_split = [<trimesh.Trimesh(vertices.shape=(3719, 3), faces.shape=(7318, 3))>, <trimesh.Trimesh(vertices.shape=(14, 3), faces.shape=(13, 3))>, <trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(11, 3))>]
meshes_split_sdf = [0.83319   0.288344  0.0107871]
removing mesh interior before segmentation
xvfb-run -n 2733 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_11388.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_11388_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/temp/fill_holes_894271.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_11388.off
removed temporary output file: /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_11388_fill_holes.off
/notebooks/Platinum_Datajoint/Neuron_Processing/temp/fill_holes_894271.mls is being deleted....
xvfb-run -n 519 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoi


meshes_split = [<trimesh.Trimesh(vertices.shape=(4148, 3), faces.shape=(8098, 3))>, <trimesh.Trimesh(vertices.shape=(142, 3), faces.shape=(276, 3))>, <trimesh.Trimesh(vertices.shape=(78, 3), faces.shape=(180, 3))>, <trimesh.Trimesh(vertices.shape=(73, 3), faces.shape=(141, 3))>, <trimesh.Trimesh(vertices.shape=(64, 3), faces.shape=(121, 3))>, <trimesh.Trimesh(vertices.shape=(49, 3), faces.shape=(81, 3))>, <trimesh.Trimesh(vertices.shape=(27, 3), faces.shape=(48, 3))>, <trimesh.Trimesh(vertices.shape=(23, 3), faces.shape=(41, 3))>, <trimesh.Trimesh(vertices.shape=(31, 3), faces.shape=(36, 3))>, <trimesh.Trimesh(vertices.shape=(17, 3), faces.shape=(19, 3))>]
meshes_split_sdf = [0.8345815  0.05554655 0.0798349  0.144692   0.110784   0.0503224
 0.175216   0.0169666  0.0920033  0.232982  ]
Soma List = [<trimesh.Trimesh(vertices.shape=(3941, 3), faces.shape=(7688, 3))>, <trimesh.Trimesh(vertices.shape=(3719, 3), faces.shape=(7318, 3))>, <trimesh.Trimesh(vertices.shape=(4148, 3), faces.shape

...


There was only one mesh found from the spine process and mesh split, returning empty array



There was only one mesh found from the spine process and mesh split, returning empty array



There was only one mesh found from the spine process and mesh split, returning empty array



Working on limb L0 branch 1
Working on limb L0 branch 2
No spines and using precomputed width
Working on limb L0 branch 3
Working on limb L0 branch 74
Working on limb L0 branch 4
Working on limb L0 branch 6
Working on limb L0 branch 73
No spines and using precomputed width
Working on limb L0 branch 75
No spines and using precomputed width
Working on limb L0 branch 5
Working on limb L0 branch 7
Working on limb L0 branch 18
Working on limb L0 branch 17
Working on limb L0 branch 79
Working on limb L0 branch 69
No spines and using precomputed width
Working on limb L0 branch 72
No spines and using precomputed width
Working on limb L0 branch 16
Working on limb L0 branch 50
Working on limb L0 branch 0
Working on limb L0 branch 85
No spines and using precomputed width
Working on limb L0 branch 78
Working on limb L0 branch 80
No spines and using precomputed width
Working on limb L0 branch 66
No spines and using precomputed width
Working on limb L0 branch 71
No spines and using precomputed widt

TypeError: float() argument must be a string or a number, not 'list'

In [ ]:
debug

In [ ]:
adapter_objects["decomposition"]

In [ ]:
"""
debugging the insert
saved path: /mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135574982852_0.pbz2
self.spec['stage'] did not have a stage key
"""

In [ ]:
import system_utils as su

In [ ]:
import trimesh_utils as tu
main_mesh = tu.load_mesh_no_processing("main_mesh")
current_soma = tu.load_mesh_no_processing("current_soma")

In [ ]:
import neuron_visualizations as nviz

In [ ]:
nviz.plot_objects(meshes=[main_mesh,current_soma],
                 meshes_colors="random")

In [ ]:
stitching_mesh = su.decompress_pickle("stitching_mesh.pbz2")

In [ ]:
import trimesh_utils as tu
tu.split(stitching_mesh)

In [ ]:
import neuron_visualizations as nviz
nviz.plot_objects(submesh)